### The below is a Fake News Detection model using the **Fake and real news dataset** on Kaggle. I used a **2 layer Bi-LSTM** followed by a Dense Layer. The model perform quite well with an **accuracy** of over **99%** 

Reading Data

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.layers import TextVectorization,Input,Dense,Embedding,SimpleRNN,LSTM,Bidirectional
from tensorflow.keras import Model
data_fake=pd.read_csv('/content/drive/MyDrive/Datasets/Fake/Fake.csv') #provide path for kaggle data set in fake category
data_true=pd.read_csv('/content/drive/MyDrive/Datasets/Fake/True.csv') #provide path for kaggle data set in true category
data_fake=np.array(data_fake[['text']])
data_fake=np.hstack((data_fake,np.zeros(data_fake.shape)))
data_fake

array([['Donald Trump just couldn t wish all Americans a Happy New Year and leave it at that. Instead, he had to give a shout out to his enemies, haters and  the very dishonest fake news media.  The former reality show star had just one job to do and he couldn t do it. As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year,  President Angry Pants tweeted.  2018 will be a great year for America! As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year. 2018 will be a great year for America!  Donald J. Trump (@realDonaldTrump) December 31, 2017Trump s tweet went down about as welll as you d expect.What kind of president sends a New Year s greeting like this despicable, petty, infantile gibberish? Only Trump! His lack of decency 

In [ ]:
data_true=np.array(data_true[['text']])
data_true=np.hstack((data_true,np.ones(data_true.shape)))
data_true

array([['WASHINGTON (Reuters) - The head of a conservative Republican faction in the U.S. Congress, who voted this month for a huge expansion of the national debt to pay for tax cuts, called himself a “fiscal conservative” on Sunday and urged budget restraint in 2018. In keeping with a sharp pivot under way among Republicans, U.S. Representative Mark Meadows, speaking on CBS’ “Face the Nation,” drew a hard line on federal spending, which lawmakers are bracing to do battle over in January. When they return from the holidays on Wednesday, lawmakers will begin trying to pass a federal budget in a fight likely to be linked to other issues, such as immigration policy, even as the November congressional election campaigns approach in which Republicans will seek to keep control of Congress. President Donald Trump and his Republicans want a big budget increase in military spending, while Democrats also want proportional increases for non-defense “discretionary” spending on programs that suppor

In [ ]:
data=np.vstack((data_true,data_fake))
np.random.shuffle(data)
data

array([['Tune in to the Alternate Current Radio Network (ACR) for another LIVE broadcast of  The Boiler Room  starting at 6 PM PST | 9 PM EST for this special broadcast. Join us for uncensored, uninterruptible talk radio, custom-made for barfly philosophers, misguided moralists, masochists, street corner evangelists, media-maniacs, savants, political animals and otherwise lovable rascals.Join ACR hosts Hesher, & Spore along with Andy Nowicki of Alt Right Blogspot, Jay Dyer from jaysanalysis.com, Jamie Hanshaw author of Weird Stuff, Branko Mali  of Kali Tribune, Randy J of 21Wire and Daniel Spaulding of Soul of the East for the 70th episode of BOILER ROOM. Dim the lights, dawn the headphones and indulge in some Boiler Room with the crew. .Please like and share the program and visit our donate page to get involved!BOILER ROOM IS NOT A POLICTALLY CORRECT ZONE!  LISTEN TO THE SHOW IN THE PLAYER BELOW   ENJOY! Reference Links:',
        0.0],
       ['WASHINGTON (Reuters) - Democrat Hillary

In [ ]:
data_train,data_test=data[:int(data.shape[0]*0.8),:],data[int(data.shape[0]*0.8):,:]

### Text Vectorization using Tensorflow (version should be 2.6 or later)

In [ ]:
encode=TextVectorization()
encode.adapt(data[:,:1])
x_train,y_train=encode(data_train[:,:1]),data_train[:,-1].astype('int32')
x_test,y_test=encode(data_test[:,:1]),data_test[:,-1].astype('int32')

### Model Creation

In [ ]:
input=Input(shape=(None,))
#x=encode(input)
x=Embedding(10000,100,input_length=512)(input)
x=Bidirectional(LSTM(64,return_sequences=True))(x)
x=Bidirectional(LSTM(64))(x)
output=Dense(1,activation='sigmoid')(x)
model=Model(input,output)

### Model Summary

In [ ]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_2 (Embedding)      (None, None, 100)         1000000   
_________________________________________________________________
bidirectional (Bidirectional (None, None, 128)         84480     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               98816     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 1,183,425
Trainable params: 1,183,425
Non-trainable params: 0
_________________________________________________________________


### Model training

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
history=model.fit(x_train,y_train,validation_split=0.2,batch_size=64,epochs=1)

449/449 [==============================] - 538s 1s/step - loss: 0.0458 - accuracy: 0.9796 - val_loss: 0.0083 - val_accuracy: 0.9978


### Model Evaluation

In [ ]:
model.evaluate(x_test,y_test)

281/281 [==============================] - 116s 413ms/step - loss: 0.0060 - accuracy: 0.9984


[0.0059800962917506695, 0.9984409809112549]